In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker

# ----------------------
# 1. LOAD DATA
# ----------------------
raw_df = pd.read_excel("vivvix_clean.xlsx") # Original dataset after a first round of cleaning in Excel
grouping = pd.read_excel("category_grouping.xlsx") # Table of categories and their assigned groups

# ----------------------
# 2. GENERATE UNIQUE FAKE COMPANY NAMES
# ----------------------
fake = Faker()
fake.unique.clear()

unique_companies = raw_df["Company"].unique()
company_map = {company: fake.unique.company() for company in unique_companies}

# Save the mapping
pd.DataFrame.from_dict(company_map, orient="index", columns=["Fake_Company"])\
    .reset_index().rename(columns={"index": "Original_Company"})\
    .to_csv("company_mapping.csv", index=False)
print("Company mapping saved to company_mapping.csv")

# ----------------------
# 3. TRANSFORM MAIN DATA
# ----------------------
vivvix = raw_df.copy()

# Map categories to groups
grouping_dict = dict(zip(grouping["Category"], grouping["Group"]))
vivvix["Group"] = vivvix["Category"].map(grouping_dict)

# Apply fake company names
vivvix["Fake_Company"] = vivvix["Company"].map(company_map)

# Add ±75% noise
def add_noise(x):
    return x * random.uniform(0.25, 1.75)

vivvix["Noisy_Spend"] = vivvix["Spend"].apply(add_noise)

# ----------------------
# 4. CALCULATE TIERS PER Fake_Company + Category + Year
# ----------------------
company_cat_year_spend = (
    vivvix.groupby(["Fake_Company", "Category", "Year"], as_index=False)
          .agg({"Noisy_Spend": "sum"})
          .rename(columns={"Noisy_Spend": "Total_Spend"})
)

# Define bins and labels
bins = [0, 50_000, 250_000, 1_000_000, 5_000_000, 20_000_000, float("inf")]
labels = [
    "Micro",
    "Small",
    "Mid",
    "Large",
    "Major",
    "Mega"
]

company_cat_year_spend["Spend_Tier"] = pd.cut(
    company_cat_year_spend["Total_Spend"],
    bins=bins,
    labels=labels,
    right=False
)

# ----------------------
# 5. MERGE BACK THE TIERS
# ----------------------
vivvix = vivvix.merge(
    company_cat_year_spend[["Fake_Company", "Category", "Year", "Total_Spend", "Spend_Tier"]],
    on=["Fake_Company", "Category", "Year"],
    how="left"
)

# ----------------------
# 6. FINAL CHECK & EXPORT
# ----------------------
if vivvix["Spend_Tier"].isna().sum() == 0:
    print("All companies successfully classified into spend tiers (per category/year).")
else:
    print("Some spend tiers are missing. Check for unmatched Fake_Company + Category + Year.")

vivvix.to_excel("vivvix_anonymized.xlsx", index=False)
print("Final file saved as: vivvix_anonymized.xlsx")

Company mapping saved to company_mapping.csv
All companies successfully classified into spend tiers (per category/year).
Final file saved as: vivvix_anonymized.xlsx
